In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

2021.07.19

# 计算位势高度的纬向、时间平均

采用和此前NCL脚本相似的方式读入数据，即ERA5I从筛选的ERA5I读入，VR\RCM数据从diag中选取（不太适合进行垂直剖面的绘制）

2022.02.07

将绘制时间改到MJJA

- 读入day of year数据

In [ ]:

filename_era5i = "ydaymean_hgt_daily.nc"
filename_vr     = "ydaymean_hgt_VR_98-17.nc"
filename_rcm    = "ydaymean_hgt_RCM_98-17.nc"

ds_or = {}
dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/hgt/"
ds_or['era5i']  = xr.open_dataset(dir_in + filename_era5i)
dir_in = "/raid52/yycheng/MPAS/VR_postprocess/VR_merge/diag_daily_selected/hgt/"
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
dir_in = "/raid52/yycheng/MPAS/RCM_postprocess/RCM_merge/diag_daily_selected/hgt/"
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var = {}
var['era5i'] = ds_or['era5i']['hgt'].sel(level=500)[30:,:,:]
var['vr'] = ds_or['vr']['height_500hPa'][30:,:,:]
var['rcm'] = ds_or['rcm']['height_500hPa'][30:,:,:]
# change coords
var_list = ['era5i', 'vr', 'rcm']
for i in var_list:
    rename_dict = dict(zip(var[i].coords.keys(), var['era5i'].coords.keys()))
    # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var[i] = var[i].rename(rename_dict)
    var[i]._coords = var['era5i']._coords
    var[i] = var[i].rename(i)


In [ ]:
region_dict = {}
lat = var['era5i'].lat
lon = var['era5i'].lon
time= var['era5i'].time
region_dict['lat'] = lat[(lat>=10) & (lat<=60)]
region_dict['lon'] = lon[(lon>=110) & (lon<=140)]
var_lattime = {}
for ivar in var.keys():
    var_lattime[ivar] = var[ivar].sel(region_dict).mean('lon')

## 绘图部分

- time-lat sections

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
# ----- get filter vars coords-----

cmap="gist_ncar"
# cmap="tab20"
# cmap = "tab20b"

lat_sel  = var_lattime['era5i'].lat
# time_sel = var_lattime['cn051'].time.dt.strftime("%m-%d")
time_sel = var_lattime['era5i'].time.values
# 500hPa
ticks = np.concatenate([np.arange(5300, 5700, 40), np.arange(5700, 5900, 10)])
ticks_contour = np.arange(5800, 5920, 20)

# 850hPa
# ticks = np.arange(1300, 1580, 20)
# ticks_contour = np.arange(1300, 1580, 20)


#----- create plot -----
shape_array = [[0,1,1,0],[2,2,3,3]]
fig, axs = plot.subplots(shape_array,figsize=(16,8))

axs[0].format(title='ERA-Interim')
axs[1].format(title='VR')
axs[2].format(title='RCM')
# m = axs[0].contour(lon_sel, time_sel, var_lattime['cn051'],\
# levels=ticks,color='k',
# linewidth = .5)
for i, ivar in enumerate(var_list):
    m_overlay = axs[i].contourf(time_sel,  lat_sel, var_lattime[ivar].values.T, cmap=cmap,\
    levels=ticks)
    m_contour = axs[i].contour(time_sel,  lat_sel, var_lattime[ivar].values.T,\
    levels=ticks_contour,linewidth=1,color='k')
    axs[i].clabel(m_contour, inline=True, fontsize=10, colors = 'k', fmt='%4.0f',use_clabeltext=True, inline_spacing=0)
# format作为整个proplot集中对属性进行修改的method
axs.format(
abc=True, abcloc='ul',
xrotation=0,  # rotated time ticks
xlabel='time',
yformatter='deglat',
ylabel='latitude',
ylim=(18,50),
xminorlocator=('day',range(0,30,5)),
xlocator=('month',range(5,12,1)),
xformatter = '%b', #title='Ticks every N units',
xgridminor = True,
suptitle="time-latitude cross-section of 500hPa mean height averaged over " + r"$110^{\circ}-140^{\circ}E$"
)

# for tick in axs.get_xticklabels():
    # tick.set_rotation(45)
#----- add contour tick labels -----
fig.colorbar(m_overlay, loc='r', length = 1., width=0.1,extend='both', label='[gpm]',
ticklabelsize=5,ticks=ticks)

fig.patch.set_facecolor('white')
plt.savefig('./output_pic/hgt_500_98-17_lat-time.2022.02.07.png', dpi=600, facecolor= "white")